<a href="https://colab.research.google.com/github/roberto0106/python_multi/blob/main/Envio_de_email.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import smtplib
from email.message import EmailMessage
from google.colab import files
import getpass
import os
from tqdm import tqdm  # biblioteca para barra de progresso

# 1. Upload do arquivo
print("Selecione o arquivo Excel para upload:")
uploaded = files.upload()
aba = input("Digite o nome da aba: ")
excel_path = next(iter(uploaded))
print(f"Arquivo carregado: {excel_path}\n")

# 2. Parâmetros de envio
email_column = 'email'  # coluna com e-mails
smtp_server = 'smtp.gmail.com'
smtp_port = 587
smtp_user = input('Digite seu email Gmail (remetente): ')
smtp_password = getpass.getpass('Digite sua App Password do Gmail (senha): ')
colunas_transpostas_input = input('Digite os nomes das colunas a transpor (separadas por vírgula): ').strip()
colunas_transpostas = [col.strip() for col in colunas_transpostas_input.split(',') if col.strip()]


# 3. Envio de teste
enviar_teste = input('Deseja enviar um e-mail de teste antes? (s/n): ').strip().lower() == 's'
email_teste = smtp_user if enviar_teste else None

# 4. Carrega e agrupa o DataFrame
print("Carregando planilha... ")
df = pd.read_excel(excel_path, sheet_name=aba)

if email_column not in df.columns:
    raise ValueError(f"Coluna de email '{email_column}' não encontrada.")

groups = df.groupby(email_column)
total = len(groups)
print(f"Encontrados {total} destinatários.\n")

# 5. Envio dos emails com barra de progresso
print("Iniciando envios...\n")
for i, (recipient, group) in enumerate(tqdm(groups, desc="Enviando relatórios", total=total)):
    # Se envio de teste, só envia para o remetente
    if enviar_teste and i > 0:
        break
    if enviar_teste:
        print("\n[TESTE] Enviando apenas para o remetente...\n")
        recipient = smtp_user

    # Ignora se email estiver vazio
    if pd.isna(recipient) or not recipient.strip():
        continue

    # Gera nome do arquivo
    safe_recipient = recipient.replace('@', '_at_').replace('.', '_')
    filename = f'report_{safe_recipient}.xlsx'
    group.to_excel(filename, index=False)

    # Monta conteúdo extra se a coluna foi especificada
    extra_content = ""

    if colunas_transpostas:
        extra_linhas = []
        for coluna in colunas_transpostas:
            if coluna in group.columns:
                valores = group[coluna].dropna().astype(str).tolist()
                for valor in valores:
                    extra_linhas.append(f"- {coluna}: {valor}")
        if extra_linhas:
            extra_content = "\n\nDetalhamento:\n" + "\n".join(extra_linhas)

    # Mensagem
    subject = f'Relatório para {recipient}'
    body = (
        f"Olá, {recipient}!\n\n"
        f"Bom dia, segue em anexo o extrato de sua premiação referente ao mês de Abril.\n"
        f"{extra_content}\n\n"
        f"Qualquer dúvida, estamos à disposição."
    )

    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = smtp_user
    msg['To'] = recipient
    msg.set_content(body)

    with open(filename, 'rb') as f:
        file_data = f.read()
    msg.add_attachment(file_data, maintype='application', subtype='vnd.openxmlformats-officedocument.spreadsheetml.sheet', filename=filename)

    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(smtp_user, smtp_password)
        server.send_message(msg)

    print(f"Email enviado para: {recipient}")

    os.remove(filename)  # limpa arquivo local após envio

    if enviar_teste:
        print("Email de teste enviado com sucesso.\n")
        break

print("\nProcesso finalizado.")


Selecione o arquivo Excel para upload:


Saving Premiações VPTECH 2025.xlsx to Premiações VPTECH 2025 (3).xlsx
Digite o nome da aba: Extrato para Envio Teste
Arquivo carregado: Premiações VPTECH 2025 (3).xlsx

Digite seu email Gmail (remetente): roberto.flor@grupomulti.com.br
Digite sua App Password do Gmail (senha): ··········
Digite o nome da coluna que será transposta no corpo do email (ou deixe em branco para ignorar): BOLINHAS
Deseja enviar um e-mail de teste antes? (s/n): S
Carregando planilha... 
Encontrados 1 destinatários.

Iniciando envios...



Enviando relatórios:   0%|          | 0/1 [00:00<?, ?it/s]


[TESTE] Enviando apenas para o remetente...



Enviando relatórios:   0%|          | 0/1 [00:02<?, ?it/s]

Email enviado para: roberto.flor@grupomulti.com.br
Email de teste enviado com sucesso.


Processo finalizado.
